[Tutorial](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-2-word-vectors). This part of the tutorial will focus on using distributed word vectors created by the Word2Vec algorithm.

In [1]:
import pandas as pd

In [2]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk.data

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Read data from files 
train = pd.read_csv("../input/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

test = pd.read_csv("../input/testData.tsv", header=0, delimiter="\t", quoting=3)

unlabeled_train = pd.read_csv("../input/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [5]:
# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews" % (train["review"].size,  
                                                                                            test["review"].size, 
                                                                                            unlabeled_train["review"].size))

Read 25000 labeled train reviews, 25000 labeled test reviews, and 50000 unlabeled reviews


# Preparing to train model

The functions we write to clean the data are also similar to Part 1, although now there are a couple of differences. 

- First, to train Word2Vec it is better not to remove stop words because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors. For this reason, we will make stop word removal optional in the functions below. 

- It also might be better not to remove numbers.

In [6]:
# Here it may be better if we remove number
def review_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, "lxml").get_text()
    #  
    # 2. Remove non-letters --> Remove non-letters/non-numbers
    review_text = re.sub("[^a-zA-Z0-9]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words and numbers (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    #
    # 5. Return a list of words
    return(words)

In [7]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [8]:
# Define a function to split a review into parsed sentences
def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [9]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set
Parsing sentences from unlabeled set


In [10]:
len(sentences)

795538

In [11]:
sentences[0]

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'i',
 've',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again']

In [12]:
sentences[1]

['maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent']

# Training and Saving model

In [14]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [15]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [17]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, 
            size=num_features, min_count = min_word_count,
            window = context, sample = downsampling)

2017-05-02 10:54:02,028 : INFO : 'pattern' package not found; tag filters are not available for English
2017-05-02 10:54:02,049 : INFO : collecting all words and their counts
2017-05-02 10:54:02,051 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-05-02 10:54:02,122 : INFO : PROGRESS: at sentence #10000, processed 227240 words, keeping 18038 word types
2017-05-02 10:54:02,186 : INFO : PROGRESS: at sentence #20000, processed 454552 words, keeping 25323 word types


Training model...


2017-05-02 10:54:02,252 : INFO : PROGRESS: at sentence #30000, processed 675250 words, keeping 30477 word types
2017-05-02 10:54:02,326 : INFO : PROGRESS: at sentence #40000, processed 902990 words, keeping 34862 word types
2017-05-02 10:54:02,398 : INFO : PROGRESS: at sentence #50000, processed 1123470 words, keeping 38328 word types
2017-05-02 10:54:02,475 : INFO : PROGRESS: at sentence #60000, processed 1346231 words, keeping 41337 word types
2017-05-02 10:54:02,548 : INFO : PROGRESS: at sentence #70000, processed 1570663 words, keeping 43985 word types
2017-05-02 10:54:02,629 : INFO : PROGRESS: at sentence #80000, processed 1791173 words, keeping 46399 word types
2017-05-02 10:54:02,706 : INFO : PROGRESS: at sentence #90000, processed 2016631 words, keeping 48868 word types
2017-05-02 10:54:02,772 : INFO : PROGRESS: at sentence #100000, processed 2239792 words, keeping 50979 word types
2017-05-02 10:54:02,846 : INFO : PROGRESS: at sentence #110000, processed 2460728 words, keeping 

In [18]:
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2017-05-02 10:56:07,833 : INFO : precomputing L2-norms of word weight vectors
2017-05-02 10:56:07,979 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2017-05-02 10:56:07,980 : INFO : not storing attribute syn0norm
2017-05-02 10:56:07,981 : INFO : not storing attribute cum_table
2017-05-02 10:56:08,782 : INFO : saved 300features_40minwords_10context


In [23]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [24]:
model.doesnt_match("I am hungry".split())

2017-05-02 11:01:19,921 : WARNING : vectors for words {'I'} are not present in the model, ignoring these words


'am'

In [25]:
model.most_similar("man")

[('woman', 0.6317477226257324),
 ('lady', 0.6066347360610962),
 ('lad', 0.5479551553726196),
 ('farmer', 0.540192723274231),
 ('millionaire', 0.5335596203804016),
 ('guy', 0.5265986919403076),
 ('monk', 0.5264548659324646),
 ('person', 0.5196757316589355),
 ('chap', 0.512682318687439),
 ('soldier', 0.5124808549880981)]

In [26]:
model.most_similar("queen")

[('princess', 0.6397346258163452),
 ('bride', 0.6060532331466675),
 ('stepmother', 0.5990087389945984),
 ('maid', 0.5953603982925415),
 ('prince', 0.5906099081039429),
 ('angela', 0.5871610641479492),
 ('maria', 0.5840234756469727),
 ('mistress', 0.581631600856781),
 ('goddess', 0.5792825222015381),
 ('belle', 0.5730113387107849)]

In [27]:
model.most_similar("awful")

[('terrible', 0.781831681728363),
 ('horrible', 0.7378334999084473),
 ('atrocious', 0.7155860662460327),
 ('dreadful', 0.6856135129928589),
 ('horrendous', 0.6822841167449951),
 ('abysmal', 0.6791336536407471),
 ('appalling', 0.6649953126907349),
 ('horrid', 0.6306585073471069),
 ('lousy', 0.6210367679595947),
 ('amateurish', 0.6181128025054932)]

In [37]:
model.most_similar("1")

[('0', 0.7278952598571777),
 ('4', 0.7199603915214539),
 ('3', 0.6588741540908813),
 ('ratio', 0.6184767484664917),
 ('2', 0.6152806282043457),
 ('7', 0.6093538403511047),
 ('zombi', 0.60548996925354),
 ('6', 0.6006932258605957),
 ('10', 0.5987155437469482),
 ('99', 0.5905357599258423)]

# Checking functions

In [28]:
r = 'Our model is capable of distinguishing differences in meaning! \
    It knows that men, women and children are more similar to each other than they are to kitchens.'

In [29]:
r.strip()

'Our model is capable of distinguishing differences in meaning!     It knows that men, women and children are more similar to each other than they are to kitchens.'

In [30]:
raw_r = tokenizer.tokenize(r.strip())

In [31]:
raw_r

['Our model is capable of distinguishing differences in meaning!',
 'It knows that men, women and children are more similar to each other than they are to kitchens.']

In [35]:
word_list = review_to_wordlist(raw_r[0], remove_stopwords=True)

In [36]:
word_list

['model', 'capable', 'distinguishing', 'differences', 'meaning']